In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import time
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA

df = pd.read_csv('data/conjunto_de_treinamento.csv', index_col=0)
df

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
id_solicitante,,,,,,,,,,,,,,,,,,,,,
1,1,10,internet,1,M,43,1,5,0,1,...,N,,0,9.0,4.0,NaN,NaN,718.0,718.0,0
2,1,5,presencial,1,M,57,2,0,0,1,...,Y,107,0,9.0,2.0,0.0,0.0,628.0,628.0,0
3,1,15,correio,1,M,50,2,2,0,1,...,Y,84,0,9.0,2.0,11.0,NaN,456.0,456.0,0
4,1,5,presencial,1,M,26,1,0,0,1,...,Y,111,0,13.0,5.0,0.0,0.0,563.0,563.0,0
5,1,10,internet,1,F,91,2,0,0,1,...,N,,0,NaN,1.0,NaN,NaN,484.0,484.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39996,1,10,internet,1,M,39,1,1,0,1,...,Y,58,0,9.0,2.0,NaN,NaN,960.0,960.0,0
39997,1,10,internet,1,M,63,1,0,0,1,...,N,,0,NaN,NaN,NaN,NaN,867.0,867.0,0
39998,1,5,presencial,1,M,36,2,1,0,0,...,Y,119,0,9.0,2.0,11.0,0.0,681.0,681.0,1


In [2]:
rows_with_null_data = df[df.isnull().any(axis=1)]
rows_with_null_data

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
id_solicitante,,,,,,,,,,,,,,,,,,,,,
1,1,10,internet,1,M,43,1,5,0,1,...,N,,0,9.0,4.0,NaN,NaN,718.0,718.0,0
3,1,15,correio,1,M,50,2,2,0,1,...,Y,84,0,9.0,2.0,11.0,NaN,456.0,456.0,0
5,1,10,internet,1,F,91,2,0,0,1,...,N,,0,NaN,1.0,NaN,NaN,484.0,484.0,1
6,1,5,internet,1,M,19,1,0,0,1,...,Y,107,0,9.0,2.0,NaN,NaN,628.0,628.0,0
8,1,10,internet,1,F,27,3,0,0,1,...,N,,0,2.0,5.0,NaN,NaN,787.0,787.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,1,15,internet,1,M,30,2,2,0,1,...,Y,12,0,9.0,2.0,NaN,NaN,187.0,187.0,0
39996,1,10,internet,1,M,39,1,1,0,1,...,Y,58,0,9.0,2.0,NaN,NaN,960.0,960.0,0
39997,1,10,internet,1,M,63,1,0,0,1,...,N,,0,NaN,NaN,NaN,NaN,867.0,867.0,0


In [3]:
df.isna().sum()

produto_solicitado                      0
dia_vencimento                          0
forma_envio_solicitacao                 0
tipo_endereco                           0
sexo                                    0
idade                                   0
estado_civil                            0
qtde_dependentes                        0
grau_instrucao                          0
nacionalidade                           0
estado_onde_nasceu                      0
estado_onde_reside                      0
possui_telefone_residencial             0
codigo_area_telefone_residencial        0
tipo_residencia                      1092
meses_na_residencia                  3003
possui_telefone_celular                 0
possui_email                            0
renda_mensal_regular                    0
renda_extra                             0
possui_cartao_visa                      0
possui_cartao_mastercard                0
possui_cartao_diners                    0
possui_cartao_amex                

In [4]:
# df = df.drop(['tipo_residencia'], axis=1)
# df = df.drop(['meses_na_residencia'], axis=1)
# df = df.drop(['profissao'], axis=1)
# df = df.drop(['ocupacao'], axis=1)
df = df.drop(['profissao_companheiro'], axis=1)
df = df.drop(['grau_instrucao_companheiro'], axis=1)
df = df.dropna(subset=['local_onde_reside'])
df = df.dropna(subset=['local_onde_trabalha'])

In [5]:
# https://pbpython.com/categorical-encoding.html
# possibilit to improve using dummies
df.select_dtypes(include=['object']).head()

,forma_envio_solicitacao,sexo,estado_onde_nasceu,estado_onde_reside,possui_telefone_residencial,codigo_area_telefone_residencial,possui_telefone_celular,vinculo_formal_com_empresa,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho
id_solicitante,,,,,,,,,,,
1,internet,M,RN,DF,Y,66,N,N,,N,
2,presencial,M,CE,CE,Y,107,N,Y,CE,Y,107
3,correio,M,AC,BA,Y,84,N,Y,BA,Y,84
4,presencial,M,PE,PE,Y,111,N,Y,,Y,111
5,internet,F,BA,BA,N,,N,N,,N,


In [6]:
for c in df.select_dtypes(include=['object']):
    df[c] = df[c].astype('category')
    df[c] = df[c].cat.codes


In [7]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [8]:
df = df.drop(['qtde_contas_bancarias_especiais'], axis=1)
df = df.drop(['local_onde_trabalha'], axis=1)
df = df.drop(['possui_telefone_celular'], axis=1)
df = df.drop(['grau_instrucao'], axis=1)

https://scikit-learn.org/stable/modules/impute.html

A more sophisticated approach is to use the IterativeImputer class, which models each feature with missing values as a function of other features, and uses that estimate for imputation. It does so in an iterated round-robin fashion: at each step, a feature column is designated as output y and the other feature columns are treated as inputs X. A regressor is fit on (X, y) for known y. Then, the regressor is used to predict the missing values of y. This is done for each feature in an iterative fashion, and then is repeated for max_iter imputation rounds. The results of the final imputation round are returned.

In the statistics community, it is common practice to perform multiple imputations, generating, for example, m separate imputations for a single feature matrix. Each of these m imputations is then put through the subsequent analysis pipeline (e.g. feature engineering, clustering, regression, classification). The m final analysis results (e.g. held-out validation errors) allow the data scientist to obtain understanding of how analytic results may differ as a consequence of the inherent uncertainty caused by the missing values. The above practice is called multiple imputation.

Our implementation of IterativeImputer was inspired by the R MICE package (Multivariate Imputation by Chained Equations) 1, but differs from it by returning a single imputation instead of multiple imputations. However, IterativeImputer can also be used for multiple imputations by applying it repeatedly to the same dataset with different random seeds when sample_posterior=True. See 2, chapter 4 for more discussion on multiple vs. single imputations.

It is still an open problem as to how useful single vs. multiple imputation is in the context of prediction and classification when the user is not interested in measuring uncertainty due to missing values.

Note that a call to the transform method of IterativeImputer is not allowed to change the number of samples. Therefore multiple imputations cannot be achieved by a single call to transform.


In [9]:
df_x = df.drop(['inadimplente'], axis=1)
df_y = df[['inadimplente']]

imp = IterativeImputer(max_iter=10, random_state=0)
df_x = imp.fit_transform(df_x)

In [10]:
pca = PCA(n_components='mle')

print('columns before', len(pd.DataFrame(df_x).columns))
df_x = pca.fit_transform(df_x)
print('columns after', len(pd.DataFrame(df_x).columns))

columns before 34
columns after 33


In [11]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.30)

clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)
print(clf.tree_.max_depth)
print(clf.score(x_test, y_test))

clf = RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200)
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

58
0.6255


0.73525

In [12]:
parameters = {
    'n_estimators':(10, 50, 100, 200),
    'max_depth': (5, 10, 20, 40, 80),
    'min_samples_split': (0.5, 1, 2, 5, 10),
}
start = time.time() 
clf = GridSearchCV(RandomForestClassifier(), parameters, cv=5, refit=True, n_jobs=5)
clf.fit(df_x, df_y)
end = time.time() 
print('total execution time:', "%.2f" % (end-start))

total execution time: 1063.18


In [13]:
print(clf.best_estimator_)
print(clf.best_score_)

RandomForestClassifier(max_depth=10, min_samples_split=10)
0.7399934929366172


In [14]:
clf.cv_results_

{'mean_fit_time': array([2.55316877e-01, 1.29435520e+00, 2.41840706e+00, 4.83655882e+00,
        3.23136330e-02, 4.24860954e-02, 8.51719856e-02, 1.32843781e-01,
        7.09501219e-01, 3.46468081e+00, 6.79953804e+00, 1.39009903e+01,
        7.13291454e-01, 3.45814905e+00, 6.99055614e+00, 1.36283949e+01,
        6.77309513e-01, 3.52071743e+00, 7.14649596e+00, 1.42253791e+01,
        2.48339272e-01, 1.21575646e+00, 2.41836925e+00, 4.76197577e+00,
        2.87222862e-02, 4.46805477e-02, 7.76005268e-02, 1.19081402e-01,
        1.22611938e+00, 6.04434662e+00, 1.30865536e+01, 2.63515254e+01,
        1.23253350e+00, 7.12011714e+00, 1.37088151e+01, 2.86808492e+01,
        1.45862007e+00, 6.70606122e+00, 1.34615011e+01, 2.69837825e+01,
        3.58771992e-01, 1.38802581e+00, 2.70154986e+00, 5.28386407e+00,
        3.45071316e-02, 5.38552284e-02, 7.72994518e-02, 1.17193508e-01,
        2.11166096e+00, 1.10564956e+01, 2.08677106e+01, 4.21652232e+01,
        2.17351308e+00, 1.14318405e+01, 2.11720

In [15]:
df = pd.read_csv('data/conjunto_de_teste.csv', index_col=0)

df = df.drop(['profissao_companheiro'], axis=1)
df = df.drop(['grau_instrucao_companheiro'], axis=1)
df = df.dropna(subset=['local_onde_reside'])
df = df.dropna(subset=['local_onde_trabalha'])

df = df.drop(['qtde_contas_bancarias_especiais'], axis=1)
df = df.drop(['local_onde_trabalha'], axis=1)
df = df.drop(['possui_telefone_celular'], axis=1)
df = df.drop(['grau_instrucao'], axis=1)

for c in df.select_dtypes(include=['object']):
    df[c] = df[c].astype('category')
    df[c] = df[c].cat.codes

imp = IterativeImputer(max_iter=10, random_state=0)
df = imp.fit_transform(df)

df = pd.DataFrame(pca.fit_transform(df))

df['inadimplente'] = clf.predict(df)
print(df['inadimplente'].value_counts())
df[['inadimplente']].to_csv('output/notebookRandomForestFeaturesInputed.csv')

0    10000
Name: inadimplente, dtype: int64
